In [17]:
seed = 1
import numpy as np

def seed_torch(seed=1029):
    np.random.seed(seed)
    torch.manual_seed(seed)



In [18]:
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import torch.nn as nn
class Net(nn.Module):
    def __init__(self, D_in, H1, H2, D_out):
 
        super(Net, self).__init__()
        self.linear1 = nn.Linear(D_in, H1) 
        self.linear2 = nn.Linear(H1, H2)
        self.linear3 = nn.Linear(H2, D_out)
  
    def forward(self, x):
    
        x = torch.sigmoid(self.linear1(x))
        x = torch.sigmoid(self.linear2(x))
        x = self.linear3(x)
        return x



In [19]:
import copy
import numpy as np
seed_torch(seed)
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H1, H2, D_out = 50, 7, 10, 3, 3
model = Net(D_in, H1, H2, D_out)
loss_fn = torch.nn.MSELoss()
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
init_state = copy.deepcopy(model.state_dict())
init_state_opt = copy.deepcopy(optimizer.state_dict())


In [21]:

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from tqdm import tqdm
import torch
import xlrd
import pandas as pd
book = xlrd.open_workbook('kinetic data for standardisation.xlsx')
sheet = book.sheet_by_name('Sheet1')
data_unreplicated = [[sheet.cell_value(r, c)
         for c in range(0,7)] for r in range(0,300)]
scaler = StandardScaler()
scaler.fit(data_unreplicated)
book = xlrd.open_workbook('prepareddatakfoldkineticnewANN.xlsx')
sheet = book.sheet_by_name('Sheet1')
replicated_data = [[sheet.cell_value(r, c)
         for c in range(0,7)] for r in range(0,15000)]

scaler = StandardScaler()
scaler.fit(data_unreplicated)
standardised_data = scaler.transform(replicated_data)

book = xlrd.open_workbook('prepareddatakfoldkinetic.xlsx')
sheet = book.sheet_by_name('Sheet1')
replicated_ydata = [[sheet.cell_value(r, c)
         for c in range(0,7)] for r in range(0,15000)]

standardised_ydata = scaler.transform(replicated_ydata)

x_train = standardised_data
y_train = standardised_ydata[:, 4:7]

n_splits = 6
splits = list(KFold(n_splits=n_splits, shuffle=False, random_state=seed)
              .split(x_train, y_train))



In [22]:
i = n_splits-1
batches = 50
for q in range (3,15,3):
    for l in range (7,35,7):
            N, D_in, H1, H2, D_out = batches, 7, l, q, 3
            model = Net(D_in, H1, H2, D_out)
            model.train()
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
            init_state = copy.deepcopy(model.state_dict())
            init_state_opt = copy.deepcopy(optimizer.state_dict())
            av_opt_epoch=0
            av_opt_loss=0
            for i, (train_idx, valid_idx) in enumerate(splits):
                model.load_state_dict(init_state)
                optimizer.load_state_dict(init_state_opt)
                x_train_fold = torch.tensor(x_train[train_idx], dtype=torch.float32)
                y_train_fold = torch.tensor(y_train[train_idx], dtype=torch.float32)
                x_val_fold = torch.tensor(x_train[valid_idx], dtype=torch.float32)
                y_val_fold = torch.tensor(y_train[valid_idx], dtype=torch.float32)
                train = torch.utils.data.TensorDataset(x_train_fold, y_train_fold)
                valid = torch.utils.data.TensorDataset(x_val_fold, y_val_fold)
                train_loader = torch.utils.data.DataLoader(train, batch_size=batches,
                                                           shuffle=True)
                valid_loader = torch.utils.data.DataLoader(valid, batch_size=batches,
                                                           shuffle=False)
                Gp=0
                epoch=0
                opt_loss=1
                while Gp<1 and epoch<300:
                    epoch+=1
                    model.train()
                    for x_batch, y_batch in train_loader:
                        y_pred = model(x_batch)
                        loss = loss_fn(y_pred, y_batch)
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                    model.eval()
                    avg_val_loss = 0.
                    for x_batch, y_batch in valid_loader:
                        y_pred = model(x_batch).detach()
                        avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader.dataset)
                    if avg_val_loss<opt_loss:
                        opt_loss = avg_val_loss  
                        opt_epoch=epoch
                    Gp=(avg_val_loss/opt_loss)-1
                av_opt_epoch+=opt_epoch/n_splits
                av_opt_loss+=opt_loss/n_splits
            print(H1,H2,av_opt_epoch,av_opt_loss)
         


7 3 176.83333333333334 0.004147379268705845
14 3 262.5 0.003928792278841138
21 3 259.8333333333333 0.0037611333819727107
28 3 265.8333333333333 0.0038591063623627025
7 6 273.5 0.003725034960731864
14 6 272.0 0.003644242619598905
21 6 257.16666666666663 0.003535091563810905
28 6 230.16666666666669 0.003672853334496419
7 9 261.33333333333337 0.003769098345562815
14 9 267.5 0.003599364001179735
21 9 219.83333333333334 0.003627196667715907
28 9 230.33333333333331 0.003537540255114436
7 12 257.5 0.0038013509129484497
14 12 287.33333333333337 0.0036693380251526823
21 12 193.00000000000003 0.0038798928702871004
28 12 256.83333333333337 0.003564352851733565
